# Prepare Files for Classification

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!tar -xf /content/drive/MyDrive/Project/ip102_v1.1.tar

In [12]:
import os
import pandas as pd
import shutil

# Create the Some folders to the folder to store the images
try:
  os.mkdir("train")
  os.mkdir("test")
  os.mkdir("val")
except Exception as e:
  pass



In [13]:
def read_actual_labels(file_name):
    # list to store the names of the image files
    file_names = []
    # List tostore the actual lables of each image
    actual_labels = []

    # Read train, test and val file to get the list of files names for each categories
    train_file = open("ip102_v1.1/"+file_name)
    for l in train_file:
        file_names.append(l.split(" ")[0])
        actual_labels.append(int(l.split(" ")[-1][:-1]))
    train_file.close()
    return file_names,actual_labels # Return the pair of the list with same size but content different


# Creating list to convert the label to actual name of categories
super_calss = ["Rice", "Corn", "Wheat", "Beet", "Alfalfa", "Vitis", "Citrus", "Mango"]
super_class_count = [ 14, 13, 9, 8, 13, 16, 19, 10]

# Create the subclass in the train, test and val folders to store images
for c in super_calss:
    try:
        os.mkdir("train/"+c)
    except Exception as e:
        continue
    try:
        os.mkdir("test/"+c)
    except Exception as e:
        continue
    try:
        os.mkdir("val/"+c)
    except Exception as e:
        continue

# This will continue previous process
class_cetegories = []
prev = 0
for i in range(0,len(super_class_count)):
    cl = list(range(prev,prev+super_class_count[i]))
    class_cetegories.append(cl)
    prev = prev + super_class_count[i]

    # Define function to map the label number ot the image name
def find_category(inp_cat):
    for i in range(len(class_cetegories)):
        if inp_cat in class_cetegories[i]:
            return super_calss[i]
        
# Function to move files to the respective folder
def movefiles(f_type,f_name,f_label):
    root_image_folder = "ip102_v1.1/images/"
    # Following for loop to move images to the trianing folder
    for i in range(len(f_name)):
        name = f_name[i]
        label = f_label[i]
        subfolder = find_category(label)  
        shutil.move(root_image_folder+name,f_type+"/"+subfolder+"/"+name)


# Create Folders for the sub classification
error = ""
try:
    os.mkdir("new_train")
    os.mkdir("new_train/train")
    os.mkdir("new_train/test")
    os.mkdir("new_train/val")
    for c in super_calss:
        os.mkdir("new_train/train/"+c)
        os.mkdir("new_train/test/"+c)
        os.mkdir("new_train/val/"+c)
except Exception as e:
    error = e

# File to Read the Actual Class of the image
classes_path = "/content/drive/MyDrive/Project/classes.txt"
dataclass = {}
class_file = open(classes_path)
for l in class_file:
    class_idx = int(l.split()[0])-1
    class_name = " ".join(l.split()[1:])
    dataclass[class_idx] = class_name
class_file.close()

# Following is the function to create the folder, 
def copy_images_to_sub_categories(r_folder,dict_file):
    r_folder = r_folder + "/"
    super_class_names = os.listdir(r_folder)
    
    for sc in super_class_names:
        for img_n in os.listdir(r_folder+sc):
            image_label = dict_file[img_n]
            sub_name = dataclass[image_label]
            if not os.path.exists("new_train/"+r_folder+"/"+sc+"/"+sub_name):
                os.mkdir("new_train/"+r_folder+"/"+sc+"/"+sub_name)
            shutil.copy(r_folder+sc+"/"+img_n,"new_train/"+r_folder+"/"+sc+"/"+sub_name+"/"+img_n)


## Move and copy files to respective folders

In [14]:
f_train,l_train = read_actual_labels("train.txt")
f_val,l_val = read_actual_labels("val.txt")

train_f = {f:l for f,l in zip(f_train,l_train)}
val_f = {f:l for f,l in zip(f_val,l_val)}

# Move files to the super categories
movefiles("train",f_train,l_train)
movefiles("val",f_val,l_val)

# Move files to the sub categories
copy_images_to_sub_categories("train",train_f)
copy_images_to_sub_categories("val",val_f)


# Import or Install Some libraries

In [15]:
!pip install barbar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
# Importing files

import torch
import os
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch.optim as optim
import argparse
from distutils.util import strtobool

from barbar import Bar
import copy
import time

In [17]:
def get_model(super_category_data,type_m):
  model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True)
  tc = len(os.listdir("new_train/"+type_m+"/"+super_category_data))
  model.classifier = nn.Sequential(model.classifier,nn.Linear(1000, tc))
  torch.cuda.empty_cache()
  model = model.to("cuda")
  return model

def add_noise(inputs):
  noise = torch.randn_like(inputs)*0.2
  return inputs + noise

# Following is to train the model
def train_model(in_model,train_set,valid_set,n_epochs):
  weight_decay =  0.00001
  val_acc_history = []
  train_acc_history = []
  val_loss_history = []
  train_loss_history = []
  dataloaders = {'train': train_set, 'val': valid_set}
  criterion = nn.CrossEntropyLoss()
  params_to_update = in_model.parameters()
  optimizer_ft = optim.Adam(params_to_update, lr= 0.0001, betas= (0.9, 0.999),
                                  eps= 1e-08, weight_decay= weight_decay)
  scheduler = optim.lr_scheduler.ExponentialLR(optimizer_ft, gamma= 0.96)
  optimizer = optimizer_ft
  is_save_checkpoint = False
  is_inception = False
  ckpepoch = 0
  since = time.time()
  best_acc = 0.0
  num_epochs = n_epochs
  device = "cuda"
  for epoch in range(ckpepoch, num_epochs):
    
      print('Epoch {}/{}'.format(epoch, num_epochs - 1))
      print('-' * 10)

      for phase in ['train', 'val']:
          if phase == 'train':
              in_model.train()
          else:
              in_model.eval()

          running_loss = 0.0
          running_corrects = 0

          for data in Bar(dataloaders[phase]):
              if len(data) > 2:
                  inputs, _ , labels = data
                  inputs = add_noise(inputs)
                  inputs = inputs.to(device)
              else:
                  inputs, labels = data
                  inputs = inputs.to(device)
              labels = labels.to(device)

              optimizer.zero_grad()

              with torch.set_grad_enabled(phase == 'train'):
                  if is_inception and phase == 'train':
                      outputs, aux_outputs = in_model(inputs)
                      loss1 = criterion(outputs, labels)
                      loss2 = criterion(aux_outputs, labels)
                      loss = loss1 + 0.4*loss2
                  else:
                      outputs = in_model(inputs)
                      loss = criterion(outputs, labels)

                  _, preds = torch.max(outputs, 1)

                  if phase == 'train':
                      loss.backward()
                      optimizer.step()

              running_loss += loss.item() * inputs.size(0)
              inputs.to("cpu")
              torch.cuda.empty_cache()
              running_corrects += torch.sum(preds == labels.data)

          epoch_loss = running_loss / len(dataloaders[phase].dataset)
          epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

          print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

          if phase == 'val' and epoch_acc > best_acc:
              best_acc = epoch_acc
              best_model_wts = copy.deepcopy(in_model.state_dict())
              # if is_save_checkpoint:
              #     torch.save({'epoch': epoch,
              #             'model_state_dict': model.state_dict(),
              #             'optimizer_state_dict': optimizer.state_dict(),
              #             'best_acc': best_acc,
              #             'scheduler_state_dict': scheduler.state_dict(),
              #             }, checkpointFn)

              count2stop = 0

          elif phase == 'val':
              count2stop += 1

          if phase == 'val':
              val_loss_history.append(epoch_loss)
              val_acc_history.append(epoch_acc)
          if phase == 'train':
              train_loss_history.append(epoch_loss)
              train_acc_history.append(epoch_acc)


      if count2stop == 10:
          break

      if scheduler:
          scheduler.step()
          print('lr :', scheduler.get_lr())

      print()

  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))

  in_model.load_state_dict(best_model_wts)
  his = {'train_loss': train_loss_history,
        'train_acc': train_acc_history,
        'val_loss': val_loss_history,
        'val_acc': val_acc_history}

In [18]:
input_size = 224
weight_decay =  0.00001
data_transforms = {
            'train': transforms.Compose([
               transforms.Resize(256),
               transforms.AugMix(),
                transforms.RandomCrop(input_size),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
            'val': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(input_size),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]),
        }

for s_cname in super_calss:
  model_d = get_model(s_cname,"train")
  batch_size = 32
  train_set = ImageFolder(root= "new_train/train/"+s_cname, transform= data_transforms['train'])
  valid_set = ImageFolder(root= "new_train/val/"+s_cname, transform= data_transforms['val'])
  train_set = DataLoader(train_set, batch_size= batch_size, shuffle= True, 
                              num_workers= 8, pin_memory=True)
  valid_set = DataLoader(valid_set, batch_size= batch_size, shuffle= False, 
                                num_workers= 8, pin_memory= True)
  train_model(model_d,train_set,valid_set,1)
  torch.cuda.empty_cache()
  torch.save(model_d.state_dict(), "/content/drive/MyDrive/Project/"+s_cname+"_vgg16")

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Epoch 0/0
----------
5043/5043: [===============================>] - ETA 10.2s
train Loss: 2.3094 Acc: 0.2504
843/843: [==============================>.] - ETA 3.0s
val Loss: 1.9765 Acc: 0.3618
lr : [9.216e-05]

Training complete in 2m 41s
Best val Acc: 0.361803


/usr/local/lib/python3.9/dist-packages/torch/optim/lr_scheduler.py:587: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Epoch 0/0
----------
8404/8404: [===============================>] - ETA 8.2s
train Loss: 1.3228 Acc: 0.5806
1399/1399: [===============================>] - ETA 2.0s
val Loss: 0.9173 Acc: 0.7162
lr : [9.216e-05]

Training complete in 4m 12s
Best val Acc: 0.716226


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


...

Epoch 0/0
----------
10525/10525: [===============================>] - ETA 6.1s
train Loss: 1.0740 Acc: 0.6780
1752/1752: [===============================>] - ETA 3.2s
val Loss: 0.7110 Acc: 0.7865
lr : [9.216e-05]

Training complete in 5m 8s
Best val Acc: 0.786530


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Epoch 0/0
----------
4356/4356: [===============================>] - ETA 7.6s
train Loss: 1.9260 Acc: 0.4318
725/725: [==============================>.] - ETA 2.3s
val Loss: 1.3592 Acc: 0.6055
lr : [9.216e-05]

Training complete in 2m 8s
Best val Acc: 0.605517


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


Epoch 0/0
----------
5840/5840: [===============================>] - ETA 9.0s
train Loss: 0.9569 Acc: 0.6998
971/971: [==============================>.] - ETA 3.4s
val Loss: 0.6102 Acc: 0.7827
lr : [9.216e-05]

Training complete in 2m 54s
Best val Acc: 0.782698
